In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.3 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split


In [7]:
# Load the IMDB dataset from CSV
df = pd.read_csv('/content/drive/MyDrive/IMDB_Dataset.csv', encoding='utf-8',on_bad_lines='skip')


In [8]:
test_df = pd.read_csv('/content/drive/MyDrive/movie-review-dataset.csv', on_bad_lines='skip')

In [9]:
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})

print(df.head(20))

                                               review  sentiment
0   One of the other reviewers has mentioned that ...          1
1   A wonderful little production. <br /><br />The...          1
2   I thought this was a wonderful way to spend ti...          1
3   Basically there's a family where a little boy ...          0
4   Petter Mattei's "Love in the Time of Money" is...          1
5   Probably my all-time favorite movie, a story o...          1
6   I sure would like to see a resurrection of a u...          1
7   This show was an amazing, fresh & innovative i...          0
8   Encouraged by the positive comments about this...          0
9   If you like original gut wrenching laughter yo...          1
10  Phil the Alien is one of those quirky films wh...          0
11  I saw this movie when I was about 12 when it c...          0
12  So im not a big fan of Boll's work but then ag...          0
13  The cast played Shakespeare.<br /><br />Shakes...          0
14  This a fantastic movi

In [10]:
# Preprocess the text column
import re
import pandas as pd
import numpy as np
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove html tags
    text = re.sub('<[^<]+?>', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

df['review'] = df['review'].apply(preprocess_text)

In [11]:
df1 = df[df['review'].str.split().str.len() < 100]

print(df1.head(10))


                                               review  sentiment
9   if you like original gut wrenching laughter yo...          1
10  phil the alien is one of those quirky films wh...          0
14  this a fantastic movie of three prisoners who ...          1
22  what an absolutely stunning movie if you have ...          1
36  the plot is about the death of little children...          0
46  protocol is an implausible movie whose only sa...          0
49  average and surprisingly tame fulci giallo whi...          0
53  i cannot believe i enjoyed this as much as i d...          1
62  so lets beginthe movie itself is as original a...          1
63  besides being boring the scenes were oppressiv...          0


In [12]:
# Define tokenizer
import transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



In [13]:
# Tokenize train dataset
import torch
# Tokenize train dataset
train_encodings = tokenizer(list(df1['review']), padding=True, truncation=True, max_length=100)
X_train = train_encodings['input_ids']
y_train = np.array(list(df1['sentiment']))
print(len(X_train))
print(len(y_train))



6510
6510


In [14]:
# Tokenize test dataset
test_encodings = tokenizer(list(test_df['review']), padding=True, truncation=True, max_length=100)
X_val = test_encodings['input_ids']
y_val = np.array(list(test_df['sentiment']))

In [15]:
from transformers import TFBertForSequenceClassification

# Load saved model
model_path = '/content/drive/MyDrive/Model/Model-PreTrained'
model = TFBertForSequenceClassification.from_pretrained(model_path)


Some layers from the model checkpoint at /content/drive/MyDrive/Model/Model-PreTrained were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Model/Model-PreTrained.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [16]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [17]:
# Train the model
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_train, "attention_mask": np.ones_like(X_train)},
    y_train
))
train_dataset = train_dataset.shuffle(len(X_train)).batch(32)

In [18]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_val, "attention_mask": np.ones_like(X_val)},
    y_val
))
val_dataset = val_dataset.batch(32)

In [19]:
# test_df.columns = ['review', 'sentiment']
# test_df.to_csv('test1.csv', index=False)
texts = np.array(test_df['review'])
labels = np.array(test_df['sentiment'])

In [29]:
print(test_df.shape)

(609, 2)


In [32]:
# test_df.columns = ['review', 'sentiment']
# test_df.to_csv('test1.csv', index=False)
import pandas as pd

df_adv1 = pd.read_csv('/content/drive/MyDrive/Adversarial_Examples/BAEGarg2019.csv', header=None, names=['review'])
# data.columns = ['review'] # Add header 'review' to the DataFrame


print(df_adv1.shape)



(609, 1)


In [33]:
df_adv1 = pd.DataFrame({'review': df_adv1['review'], 'sentiment': labels })

In [35]:
print(df_adv1.head(10))

                              review  sentiment
0                This movie was bad!          1
1               The acting was great          0
2               I loved this feeling          1
3            It was a moment of time          0
4                 The plot was great          0
5                This is a bad movie          1
6  I please not recommend this movie          0
7        The cinematography was used          1
8               The script was great          0
9               The ending was given          0


In [36]:
df_adv1['review'] = df_adv1['review'].apply(preprocess_text)

In [37]:
adv_df1 = df_adv1[df_adv1['review'].str.split().str.len() < 100]

In [38]:
print(adv_df1.head(10))

                              review  sentiment
0                 this movie was bad          1
1               the acting was great          0
2               i loved this feeling          1
3            it was a moment of time          0
4                 the plot was great          0
5                this is a bad movie          1
6  i please not recommend this movie          0
7        the cinematography was used          1
8               the script was great          0
9               the ending was given          0


In [42]:
# Create DataFrame with adv_texts and labels
df_adv2 = pd.read_csv('/content/drive/MyDrive/Adversarial_Examples/CheckList2020.csv', header=None, names=['review'])
df_adv2 = pd.DataFrame({'review': df_adv2['review'], 'sentiment': labels })
df_adv2['review'] = df_adv2['review'].apply(preprocess_text)
adv_df2 = df_adv2[df_adv2['review'].str.split().str.len() < 100]

In [43]:
# Create a new dataframe containing both original and adversarial examples
DF_ADV1 = pd.concat([adv_df1, adv_df2], ignore_index=True)

In [44]:
print(DF_ADV1.head(10))

                              review  sentiment
0                 this movie was bad          1
1               the acting was great          0
2               i loved this feeling          1
3            it was a moment of time          0
4                 the plot was great          0
5                this is a bad movie          1
6  i please not recommend this movie          0
7        the cinematography was used          1
8               the script was great          0
9               the ending was given          0


In [45]:
# Create DataFrame with adv_texts and labels
df_adv3 = pd.read_csv('/content/drive/MyDrive/Adversarial_Examples/DeepWordBugGao2018.csv', header=None, names=['review'])
df_adv3 = pd.DataFrame({'review': df_adv3['review'], 'sentiment': labels })
df_adv3['review'] = df_adv3['review'].apply(preprocess_text)
adv_df3 = df_adv3[df_adv3['review'].str.split().str.len() < 100]

In [46]:
DF_ADV2 = pd.concat([DF_ADV1, adv_df3], ignore_index=True)

In [47]:
print(DF_ADV2.tail(5))

                                                 review  sentiment
1822  the special effects were fmpressive and added ...          1
1823  i found the move to be emotionally reosnant an...          1
1824             the actign was emdiocre and frgettable          0
1825  thje pacing of the flim was perect and kept me...          1
1826  i pthought the mvoie was a complete wate of tvime          0


In [48]:
# Create DataFrame with adv_texts and labels
df_adv4 = pd.read_csv('/content/drive/MyDrive/Adversarial_Examples/IGAWang2019.csv', header=None, names=['review'])
df_adv4 = pd.DataFrame({'review': df_adv4['review'], 'sentiment': labels })
df_adv4['review'] = df_adv4['review'].apply(preprocess_text)
adv_df4 = df_adv4[df_adv4['review'].str.split().str.len() < 100]

In [49]:
DF_ADV3 = pd.concat([DF_ADV2, adv_df4], ignore_index=True)

In [50]:
print(DF_ADV3.tail(5))

                                                 review  sentiment
2431  the specialised effects were sumptuous and add...          1
2432  i revealing the cinemas to be mentally resonan...          1
2433     the response was unimaginative and forgettable          0
2434  the pacing of the theater was irreproachable a...          1
2435  i thought the movie was a complete residue of ...          0


In [51]:
# Create DataFrame with adv_texts and labels
df_adv5 = pd.read_csv('/content/drive/MyDrive/Adversarial_Examples/InputReductionFeng2018.csv', header=None, names=['review'])
df_adv5 = pd.DataFrame({'review': df_adv5['review'], 'sentiment': labels })
df_adv5['review'] = df_adv5['review'].apply(preprocess_text)
adv_df5 = df_adv5[df_adv5['review'].str.split().str.len() < 100]

In [52]:
DF_ADV4 = pd.concat([DF_ADV3, adv_df5], ignore_index=True)

In [53]:
# Create DataFrame with adv_texts and labels
df_adv6 = pd.read_csv('/content/drive/MyDrive/Adversarial_Examples/PWWSRen2019.csv', header=None, names=['review'])
df_adv6 = pd.DataFrame({'review': df_adv6['review'], 'sentiment': labels })
df_adv6['review'] = df_adv6['review'].apply(preprocess_text)
adv_df6 = df_adv6[df_adv6['review'].str.split().str.len() < 100]

In [54]:
DF_ADV5 = pd.concat([DF_ADV4, adv_df6], ignore_index=True)

In [55]:
# Create DataFrame with adv_texts and labels
df_adv7 = pd.read_csv('/content/drive/MyDrive/Adversarial_Examples/Pruthi2019.csv', header=None, names=['review'])
df_adv7 = pd.DataFrame({'review': df_adv7['review'], 'sentiment': labels })
df_adv7['review'] = df_adv7['review'].apply(preprocess_text)
adv_df7 = df_adv7[df_adv7['review'].str.split().str.len() < 100]

In [56]:
DF_ADV6 = pd.concat([DF_ADV5, adv_df7], ignore_index=True)

In [57]:
print(DF_ADV6.tail(10))

                                                 review  sentiment
4253  the pacihg was off and made the movie feel dis...          0
4254         i was thoroughly entrrtained by this movie          1
4255               the dixlogue was stilted and awkward          0
4256  the cinematography was breatgtaking and added ...          1
4257  the story was capitvating and kept me engaged ...          1
4258  the special effects were inpressive and added ...          1
4259  i found the move to be emotionally resonant an...          1
4260            the actkng was mediocre and forgettable          0
4261  the pacing of the film was peefect and kept me...          1
4262   i thought the movie was a complete wasye of time          0


In [58]:
# Create DataFrame with adv_texts and labels
df_adv8 = pd.read_csv('/content/drive/MyDrive/Adversarial_Examples/TextFoolerJin2019.csv', header=None, names=['review'])
df_adv8 = pd.DataFrame({'review': df_adv8['review'], 'sentiment': labels })
df_adv8['review'] = df_adv8['review'].apply(preprocess_text)
adv_df8 = df_adv8[df_adv8['review'].str.split().str.len() < 100]

In [59]:
DF_ADV7 = pd.concat([DF_ADV6, adv_df8], ignore_index=True)

In [60]:
DF_ADV7.to_csv('/content/drive/MyDrive/Adversarial_Examples/totalset_adv_examples.csv', index=False)

In [61]:
print(DF_ADV7.head(10))

                              review  sentiment
0                 this movie was bad          1
1               the acting was great          0
2               i loved this feeling          1
3            it was a moment of time          0
4                 the plot was great          0
5                this is a bad movie          1
6  i please not recommend this movie          0
7        the cinematography was used          1
8               the script was great          0
9               the ending was given          0


In [62]:
import torch
# Tokenize train dataset
train_encodings = tokenizer(list(DF_ADV7['review']), padding=True, truncation=True, max_length=100)
X_train1 = train_encodings['input_ids']
y_train1 = np.array(list(DF_ADV7['sentiment']))

In [63]:
# Train the model
train_dataset1 = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_train1, "attention_mask": np.ones_like(X_train1)},
    y_train1
))
train_dataset1 = train_dataset1.shuffle(len(X_train1)).batch(32)

In [64]:
model.fit(train_dataset1, epochs=3, batch_size=32)

Epoch 1/3
153/153 [==============================] - 93s 231ms/step - loss: 0.3678 - accuracy: 0.8319
Epoch 2/3
153/153 [==============================] - 28s 182ms/step - loss: 0.1532 - accuracy: 0.9407
Epoch 3/3
153/153 [==============================] - 28s 183ms/step - loss: 0.0702 - accuracy: 0.9770


In [65]:
model.save_pretrained('/content/drive/MyDrive/Adversarial_Examples/Accuracy-BertModel')

In [66]:
# Evaluate the model on the validation dataset
eval_results = model.evaluate(val_dataset)

# Print the accuracy
accuracy = eval_results[1]

print(f"Validation accuracy:, {accuracy*100:.4f}%")

20/20 [==============================] - 4s 59ms/step - loss: 0.0157 - accuracy: 0.9951
Validation accuracy:, 99.5074%
